Works only with work/CMSSW_10_2_3/src

In [1]:
import os, re, json
import pickle
import commands
from glob import glob
import numpy as np
from prettytable import PrettyTable

In [2]:
!export PATH=$HOME/.local/bin:/cvmfs/cms-bril.cern.ch/brilconda/bin:$PATH

# Input data

In [8]:
loc = os.environ['HOME'] + '/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp'
loc += '/crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_191126'
print loc
loc = glob(loc)
print len(loc)
tag = 'B2DstMu_191126'

/storage/user/ocerri/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp/crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_191126
1


In [12]:
fname = '../data/cmsRD/lumiReport/lumiPerTrigger_'+tag+'.pickle'
recreate = True
if os.path.isfile(fname) and not recreate:
    lumiInfo = pickle.load(open(fname, 'rb'))
else:
    lumiInfo = {}
    for main_dir in loc:
        print main_dir.split('/')[-1]
        s = main_dir.split('/')[-1]
        idx = s.find('Run2018')
        era = s[idx+len('Run2018')]
        idx = s.find('ParkingBPH')
        part = int(s[idx+len('ParkingBPH')])
        print era, part

        if not era in lumiInfo.keys():
            lumiInfo[era] = {}

        cmd = 'brilcalc lumi -u /fb --precision 2f'
        cmd += ' --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json'
        cmd += ' --hltpath HLT_Mu\*_IP?_part' + str(part-1) + '_v?'
        cmd += ' -i {}/processedLumis.json'.format(main_dir + '/results')
        if os.uname()[1] == 'login-1.hep.caltech.edu':
            cmd += ' -c web'

        status, output = commands.getstatusoutput(cmd)
        aux = output.split('#Summary:')[1]
        aux = aux.split('\n')
        for l in aux:
            if not l.startswith('| HLT'):
                continue
            l = l[1:-1]
            content = l.split(' | ')
            trgPath = content[0].replace(' ', '')
            if not '_part'+str(part-1) in trgPath:
                continue
            trgPath = trgPath.replace('_part' + str(part-1), '')

            lumi = float(content[-1].replace(' ', ''))
            if lumi < 0.02:
                continue
            if not trgPath in lumiInfo[era].keys():
                lumiInfo[era][trgPath] = np.zeros(6)
            lumiInfo[era][trgPath][part-1] = lumi
            print trgPath, part, lumi
    pickle.dump(lumiInfo, open(fname, 'wb'))

crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_191126
D 1
HLT_Mu12_IP6_v2 1 5.25
HLT_Mu7_IP4_v2 1 1.27
HLT_Mu8_IP3_v3 1 0.08
HLT_Mu8_IP5_v2 1 1.65
HLT_Mu8_IP6_v2 1 1.65
HLT_Mu9_IP5_v2 1 3.33
HLT_Mu9_IP6_v3 1 4.12


In [13]:
eras = list(np.sort(lumiInfo.keys()))
table = PrettyTable()

table.field_names = ['HLT path'] + eras

printed_paths = []
for e in eras:
    for t in lumiInfo[e]:
        t = t[:-3]
        if t in printed_paths:
            continue
        printed_paths.append(t)
        row = [t]
        for e in eras:
            if t in [path[:-3] for path in lumiInfo[e].keys()]:
                for path in lumiInfo[e].keys():
                    if t == path[:-3]:
                        row.append(str(lumiInfo[e][path]))
                        break
            else:
                row.append(' - ')
        table.add_row(row)
print table

+--------------+---------------------------------+
|   HLT path   |                D                |
+--------------+---------------------------------+
| HLT_Mu8_IP3  | [0.08 0.   0.   0.   0.   0.  ] |
| HLT_Mu9_IP5  | [3.33 0.   0.   0.   0.   0.  ] |
| HLT_Mu8_IP6  | [1.65 0.   0.   0.   0.   0.  ] |
| HLT_Mu7_IP4  | [1.27 0.   0.   0.   0.   0.  ] |
| HLT_Mu12_IP6 | [5.25 0.   0.   0.   0.   0.  ] |
| HLT_Mu8_IP5  | [1.65 0.   0.   0.   0.   0.  ] |
| HLT_Mu9_IP6  | [4.12 0.   0.   0.   0.   0.  ] |
+--------------+---------------------------------+


In [14]:
eras = list(np.sort(lumiInfo.keys()))
table = PrettyTable()

l = ['pT', 'IP']
for e in eras:
    l += [e, 'Tot ' + e]
l += ['Tot']
table.field_names = l

printed_paths = []
for e in eras:
    for t in lumiInfo[e]:
        tot_lumi = 0
        t = t[:-3]
        if t in printed_paths:
            continue
        printed_paths.append(t)
        row = [t[6:-4], t[-1]]
        for e in eras:
            if t in [path[:-3] for path in lumiInfo[e].keys()]:
                for path in lumiInfo[e].keys():
                    if t == path[:-3]:
                        row.append(str(lumiInfo[e][path]))
                        
                        s = np.sum(lumiInfo[e][path])
                        tot_lumi += s
                        row.append('{:.2f}'.format(s))
                        break
            else:
                row.append(' - ')
                row.append(' - ')
        row.append('{:.2f}'.format(tot_lumi))
        table.add_row(row)
print table

+----+----+---------------------------------+-------+------+
| pT | IP |                D                | Tot D | Tot  |
+----+----+---------------------------------+-------+------+
| 8  | 3  | [0.08 0.   0.   0.   0.   0.  ] |  0.08 | 0.08 |
| 9  | 5  | [3.33 0.   0.   0.   0.   0.  ] |  3.33 | 3.33 |
| 8  | 6  | [1.65 0.   0.   0.   0.   0.  ] |  1.65 | 1.65 |
| 7  | 4  | [1.27 0.   0.   0.   0.   0.  ] |  1.27 | 1.27 |
| 12 | 6  | [5.25 0.   0.   0.   0.   0.  ] |  5.25 | 5.25 |
| 8  | 5  | [1.65 0.   0.   0.   0.   0.  ] |  1.65 | 1.65 |
| 9  | 6  | [4.12 0.   0.   0.   0.   0.  ] |  4.12 | 4.12 |
+----+----+---------------------------------+-------+------+


In [17]:
np.sum(lumiInfo['D']['HLT_Mu9_IP6_v3'])/np.sum(lumiInfo['D']['HLT_Mu12_IP6_v2'])

0.7847619047619048